# Fast Widgets
Collection of easy to use widgets

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#Export

#Internal dependencies
from exp.nb_AudioCommon import * 

In [3]:
#Export
import ipywidgets as widgets
from fastai.vision import Image
from fastai.datasets import datapath4file
from fastai.datasets import untar_data
from IPython.display import Audio
import torch

## Support functions

In [4]:
#Export
def getTfmOrder(o):
    if hasattr(o, 'tfm'):
        tfm = o.tfm
        if hasattr(tfm, 'order'):
            return o.tfm.order;
    return None

In [5]:
def test_getTfmOrder():
    s1 = 'asd'
    assert None == getTfmOrder(s1)
    
    class TestTfm: order = 3
    class TestClass: tfm = TestTfm
        
    s2 = TestClass()
    assert 3 == getTfmOrder(s2)
    
    print('DONE')
    
test_getTfmOrder()

DONE


## Common Widgets

### Label

In [6]:
#Export
def fastLabel(text:str, text_align='center', background_color='lightgray', color='black'):
    text_align = text_align.lower()
    #HACK! ipywidget override padding...
    if 'left'==text_align: text = '&nbsp ' + text
    if 'right'==text_align: text = text + ' &nbsp'
        
    style = f'"background-color:{background_color};color:{color};text-align:{text_align};"'
    html = f'<div style={style}>{text}</div>'
    return widgets.HTML(value=html, layout=widgets.Layout(width='auto', height='auto', margin='0 0 0 0'))

In [7]:
def test_fastLabel():
    display(fastLabel('DEFAULT'))
    display(fastLabel('NO BACKGROUND', background_color='none'))
    display(fastLabel('NO BACKGROUND', background_color=None))
    display(fastLabel('WHITE ON RED', color='white', background_color='RED')) # case does not matters...
    display(fastLabel('LEFT', text_align='left'))
    display(fastLabel('RIGHT', text_align='right'))
    
test_fastLabel()

HTML(value='<div style="background-color:lightgray;color:black;text-align:center;">DEFAULT</div>', layout=Layo…

HTML(value='<div style="background-color:none;color:black;text-align:center;">NO BACKGROUND</div>', layout=Lay…

HTML(value='<div style="background-color:None;color:black;text-align:center;">NO BACKGROUND</div>', layout=Lay…

HTML(value='<div style="background-color:RED;color:white;text-align:center;">WHITE ON RED</div>', layout=Layou…

HTML(value='<div style="background-color:lightgray;color:black;text-align:left;">&nbsp LEFT</div>', layout=Lay…

HTML(value='<div style="background-color:lightgray;color:black;text-align:right;">RIGHT &nbsp</div>', layout=L…

### Card

In [8]:
#Export
def fastCard(title:str, w=None, hasBorder=True, title_background_color='lightgray', color='black', text_align='center'):
    '''Returns a widget that "wraps" the widget <w> with a box'''
    props = dict(display='flex',
                flex_flow='column',
                align_items='stretch',
                width='100%',
                grid_gap='0px 0px')
    if hasBorder: 
        props['border']='solid'
    box_layout = widgets.Layout(**props)
    
    lblw = fastLabel(title, background_color=title_background_color, text_align=text_align, color=color)
    
    if w: 
        return widgets.VBox(children=[lblw,w], layout=box_layout)
    else:
        return widgets.VBox(children=[lblw], layout=box_layout)        

In [9]:
def test_fastCard():
    out = fastCard('hello',widgets.Label('world'))
    display(out)
    # No border
    display(fastCard('hello - no border',widgets.Label('world'), hasBorder=False))
    # No content
    display(fastCard('hello - no elements inside',None))
    # Style
    display(fastCard('white on red',widgets.Label('world'), text_align='left', title_background_color='red', color='white'))
    display(fastCard('orange on blue',widgets.Label('world'), text_align='right', title_background_color='blue', color='orange'))
    display(fastCard('blue on transparent',widgets.Label('world'), text_align='center', title_background_color='none', color='blue'))
    
test_fastCard()

### Image

In [10]:
#Export
def getImageWidget(img):
    out = widgets.Output()
    with out:
        display(Image(img))
    return out

In [11]:
def test_getImageWidget():
    img = torch.zeros((1,100,200))
    out = getImageWidget(img)
    display(out)
    
test_getImageWidget()

Output()

### Transforms list

In [12]:
#Export 
def getTransformsWidget(tfms):
    items = []
    for i,t in enumerate(tfms):
        order = getTfmOrder(t)
        s = f'({i} : {order})' if order else f'({i})'
        items.append(widgets.Label(f'{s}\t' + str(t)))
    return widgets.VBox(children=items)    

In [13]:
def test_getTransformsWidget():
    tfms = ['abcd','cdef','zzz']
    display(getTransformsWidget(tfms))
    display(fastCard('TFMS',getTransformsWidget(tfms)))

    class TestTfm1: order = 3
    class TestClass1: tfm = TestTfm1
    class TestClass2: tfm = None
    a=TestClass1
    b=TestClass2
    display(getTransformsWidget([a,b]))
    
test_getTransformsWidget()

### Result and Tfms

In [14]:
#Export
def getResultAndTfmsWidget(result, tfms):
    wTfms = getTransformsWidget(tfms)
    tabs = widgets.Tab(children=[result,wTfms])
    tabs.set_title(0,'RESULT')
    tabs.set_title(1,'TFMS')
    return tabs

In [15]:
def test_getResultAndTfmsWidget():
    result = fastCard('TEST WIDGET',widgets.Label('Result of computation'))
    tfms = ['abcd','cdef','zzz']
    display(getResultAndTfmsWidget(result, tfms))
    
test_getResultAndTfmsWidget()

### Audio Widgets

In [16]:
# Download data in fast.ai format (thnx @Thom)
data_url = 'http://www.openslr.org/resources/45/ST-AEDS-20180100_1-OS'
path = datapath4file('ST-AEDS-20180100_1-OS')
untar_data(data_url, dest = path)
sampleFile = path.ls()[4]
sampleSound = AudioData.load(sampleFile)

In [17]:
#Export
def getAudioWidget(ad:AudioData, lbl=None, text_align='left'):
    out = widgets.Output()
    if ad:
        with out: 
            display(Audio(ad.sig,rate=ad.sr))
    # if None = ad => nothing to display
    if lbl: out = widgets.VBox(children=[fastLabel(lbl,background_color='none',text_align=text_align),out])
    return out

In [18]:
def test_getAudioWidget():
    out = getAudioWidget(sampleSound)
    display(out)
    # With Label
    display(getAudioWidget(sampleSound,'Test'))
    # Nothing to display
    display(getAudioWidget(None))

test_getAudioWidget()

Output()

Output()

# Export

In [19]:
!python notebook2script.py FastWidgets.ipynb

Converted FastWidgets.ipynb to nb_FastWidgets.py
